# Data Wrangling 
## Shaping 21st Century AI. Controversies and Closure in Media, Policy, and Research' project data curation process notebook

#### Content:
- Reading data
- Feature engineering and pre-processing
- Missing values
- Duplicates
- Distribution of the articles by date
- Distribution of the articles by media
- Cleaning the author column values
- Unique ID column
- Cleaning the title column values
- Text cleaning
- Visualization of the word usage

In [68]:
# Working with data
import pandas as pd
pd.options.display.max_colwidth = 3000
import random
from unidecode import unidecode
import re

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import plotly.express as px

# Text analysis
import nltk  
stopword_list = nltk.corpus.stopwords.words('french')

# NLP
import spacy.cli
nlp = spacy.load("fr_core_news_sm")

# Hide warnings 
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Text
import nltk  
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
set(stopwords.words('french'))

# NLP
import spacy.cli
nlp = spacy.load("fr_core_news_sm")

## Read data

In [69]:
# corpus with key words in the title
corpus_title = pd.read_csv('Corpus_Title.csv')
# corpus with key words in the lead paragraph
corpus_lead = pd.read_csv('Corpus_Lead.csv')

In [70]:
print('Total number of records in the title dataset:', corpus_title.shape)

Total number of records in the title dataset: (12783, 6)


In [71]:
print('Total number of records in the lead dataset:', corpus_lead.shape)

Total number of records in the lead dataset: (35628, 6)


## Feature engineering and pre-processing

In [72]:
# creating new column which indicate the key words search
corpus_title['key_words'] = 'title'
corpus_lead['key_words'] = 'lead'

In [73]:
# merging two datasets
corpus = corpus_title.append(corpus_lead)

In [74]:
# saving datasets as csv file
corpus.to_csv('corpus_full.csv', index = False, header=True)

In [75]:
print('Total number of records in the dataset:', corpus.shape)

Total number of records in the dataset: (48411, 7)


### Missing values

In [76]:
# detecting the number of NA articles
corpus['content'].isna().sum()

262

In [77]:
# deleting the NA datapoints from the dataset
corpus_complet = corpus[corpus['content'].notna()]

### Duplicates

In [78]:
# detecting the duplicates in the content column 
corpus_complet[corpus_complet.duplicated(['content'], keep=False)]

,content,author,journal,title,date,ID,key_words
273,Business WireCognigy et Telekom s’associent dans le domaine de l’IA conversationnelleLire l'article complet 17 décembre 2020 à 1:38 PM·5 min de lecture Cognigy.AI intègre la... Voir l'article Ce document référence un lien URL de site non hébergé par Cision.,NaN,Yahoo!-France-(site-web-réf.)---Yahoo-France,Cognigy-et-Telekom-s’associent-dans-le-domaine-de-l’IA-conversationnelle,2020-12-17,Cognigy-et-Telekom-s’associent-dans-le-domaine-de-l’IA-conversationnelleYahoo!-France-(site-web-réf.)---Yahoo-France2020-12-17,title
274,Business WireCognigy et Telekom s’associent dans le domaine de l’IA conversationnelleLire l'article complet 17 décembre 2020 à 1:38 PM·5 min de lecture Cognigy.AI intègre la... Voir l'article Ce document référence un lien URL de site non hébergé par Cision.,NaN,Yahoo!-Finance-France-(site-web-réf.)---Yahoo!-Finance-(FR),Cognigy-et-Telekom-s’associent-dans-le-domaine-de-l’IA-conversationnelle,2020-12-17,Cognigy-et-Telekom-s’associent-dans-le-domaine-de-l’IA-conversationnelleYahoo!-Finance-France-(site-web-réf.)---Yahoo!-Finance-(FR)2020-12-17,title
305,Cet article est paru dans Acteurs Publics,NaN,Acteurs-Publics,Le-gouvernement-incite-les-administrations-à-expérimenter-l'intelligence--artificielle,2018-06-14,Le-gouvernement-incite-les-administrations-à-expérimenter-l'intelligence--artificielleActeurs-Publics2018-06-14,title
306,Cet article est paru dans Acteurs Publics,NaN,Acteurs-Publics,Rapprochement-franco-britannique-sur-l'IA-et-l'open-source,2018-07-05,Rapprochement-franco-britannique-sur-l'IA-et-l'open-sourceActeurs-Publics2018-07-05,title
307,Cet article est paru dans Europe 1 (site web),Europe1-.fr-avec-AFP,Europe-1-(site-web),Google-inaugure-un-laboratoire-d'intelligence--artificielle-à-Paris,2018-09-18,Google-inaugure-un-laboratoire-d'intelligence--artificielle-à-ParisEurope-1-(site-web)2018-09-18,title
...,...,...,...,...,...,...,...
34649,"« Imaginez : vous faites votre jogging le dimanche. Soudain : aïe ! Douleur derrière la cuisse ! » Voici le speech de Dr Sport, la nouvelle application mobile destinée aux sportifs. Créée à l'initiative d'un médecin du sport, l'appli peut identifier votre blessure. Il suffit de répondre à cinq questions renseignant le sport que vous pratiquiez, l'endroit où vous avez mal ou encore l'intensité de votre douleur, et le diagnostic tombe. Premiers gestes à faire, types de médicaments (délivrés sans ordonnance) à se procurer et géolocalisation des professionnels de la santé se trouvant à proximité... Dr Sport vous conseille sur le type de soins à adopter. Son algorithme évalue également la gravité de la lésion et vous propose les numéros des services d'urgence à contacter en cas de besoin.",NaN,Le-Parisien--Edition-Principale;-Paris;-Oise;-Seine-et-Marne;-Essonne;-Yvelines;-Val-d'Oise;-Val-de-Marne;-Seine-St-Denis;-Hauts-de-Seine,Mon-médecin-du-sport-personnel,2016-10-03,Mon-médecin-du-sport-personnelLe-Parisien--Edition-Principale;-Paris;-Oise;-Seine-et-Marne;-Essonne;-Yvelines;-Val-d'Oise;-Val-de-Marne;-Seine-St-Denis;-Hauts-de-Seine2016-10-03,lead
35245,"À l'heure de l'intelligence artificielle, KONE bouleverse le marché des ascenseurs et des escalators grâce à des services connectés 24/7. Une technologie de pointe, qui s'avère particulièrement intéressante pour des centres commerciaux en pleine réinvention. Dans les grands magasins et les centres commerciaux, la gestion des flux est primordiale. Une expérience client réussie encourage l'acte d'achat et la satisfaction des clients implique des déplacements confortables. À l'heure du retail 2.0 où les magasins physiques se réinventent pour être plus attractifs, une panne d'escalator qui congestionne les accès aux boutiques déprécie la satisfaction du visiteur. À l'écoute des problématiques de ses clients retailers, KONE a donc développé de nouveaux services connectés totalement révolutionnaires. Grâce à des capteurs connectés installés sur les ascenseurs, l'entrepr

In [79]:
# deleting the duplicates
corpus_complet.drop_duplicates(subset ="content", keep = False, inplace = True)

<ipython-input-79-5e2fcf63f718>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [80]:
print('Total number of records in the cleaned dataset:', corpus_complet.shape)

Total number of records in the cleaned dataset: (47175, 7)


### Distribution of the articles by date

In [81]:
# creating timestamps and new varibles for the timeframe analysis
corpus_complet['date_time'] = pd.to_datetime(corpus_complet['date'])

<ipython-input-81-7e70b643ab40>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [132]:
# creating the distribution dataframe by month
dist_articles = corpus_complet['date_time'].value_counts(ascending=True).to_frame().reset_index().rename(columns={'index': 'date', 'date_time': 'count'})

In [134]:
# saving data for dashboard
dist_articles.to_csv('dist_articles.csv', index=False)

In [83]:
# ploting the distribution
fig = px.histogram(dist_articles, x='date', y='count', template='plotly_white', range_x=['2011','2021'])
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='Articles Count')
fig.update_traces(xbins_size="M1")
fig.show()

### Distribution of the articles by media

In [135]:
# display total number of unique media names
corpus_complet['journal'].nunique()

7204

In [85]:
# transforming media names to the same format
corpus_complet['journal_clean'] = corpus_complet['journal'].str.replace("--", '')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace("(", '')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(")", '')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace("-site-web", '')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(".fr", '')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(".com", '')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace("-", ' ')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'\,.+','')

corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Echos).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Monde).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Tribune).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=AGEFI).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Parisien).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Sud Ouest).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=La Provence).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Paris Normandie).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Le Huffington Post).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Presse Océan).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=La Croix).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Ouest France).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Midi Libre).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Le Progrès Lyon).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Corse Matin).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=La Montagne).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Républicain).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=La République du Centre).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Indépendant).*$','')
corpus_complet['journal_clean'] = corpus_complet['journal_clean'].str.replace(r'(?<=Le Courrier de lOuest).*$','')

<ipython-input-85-f099407db127>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-85-f099407db127>:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.

<ipython-input-85-f099407db127>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-85-f099407db127>:4: FutureWarning:

The default value of regex will change from True to False in 

<ipython-input-85-f099407db127>:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-85-f099407db127>:25: FutureWarning:

The default value of regex will change from True to False in a future version.

<ipython-input-85-f099407db127>:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-85-f099407db127>:26: FutureWarning:

The default value of regex will change from True to False in a future version.

<ipython-input-85-f099407db127>:26: SettingWithCopyWarning:


A value is trying to b

In [86]:
# total number of media sources after cleaning
corpus_complet['journal_clean'].nunique()

708

In [136]:
# creating the media distribution dataframe by number of articles published
dist_media = corpus_complet['journal_clean'].value_counts().to_frame('count').reset_index().rename(columns={'index': 'media'})

In [ ]:
# saving data for dashboard
dist_media.to_csv('dist_media.csv', index=False)

In [88]:
# media distribution visualization
fig = px.histogram(dist_media[:10], x='count', y='index', template='plotly_white')
fig.update_xaxes(title_text='Number of articles published from 2011 to 2021')
fig.update_yaxes(title_text='Media')
fig.update_traces(xbins_size="M1")
fig.show()

### Cleaning the author column values

In [89]:
# cleaning the author values
corpus_complet['author'] = corpus_complet['author'].str.replace("-", ' ')

<ipython-input-89-25c5a6ce48b1>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [90]:
# checking number of unique authors
corpus_complet['author'].nunique()

8795

In [91]:
# creating dataframe with authors names and number of articles written
authors_media = corpus_complet.value_counts(["author", "journal"])
authors_media = authors_media.reset_index().rename(columns={'index': 'author', 0:'count'})

In [92]:
# saving the names of the articles' authors for the Twitter analysis
authors_media.to_excel('media_authors.xlsx', index = False, header=True, encoding='utf-8')

### Cleaning the title column values

In [93]:
corpus_complet['title']

198                                 IA-contre-DC:-le-match-n’aura-pas-lieu
199                                     IA-:-comment-la-Chine-veut-briller
200                   [l'intelligence--artificielle-est-l'ensemble-des...]
201      L’avocat-sera-t-il-remplacé-par-une--intelligence--artificielle-?
202           [watson-et-les-banquesL'intelligence--artificielle-d'IBM...]
                                       ...                                
35623                                      Des-subventions-qui-interrogent
35624                                     Je-vends-(ou-transmets)-mon-bien
35625                      J’achète-un-logement-pour-l’occuper-ou-le-louer
35626                            Les-inquiétudes-d’un-ancien-élu-noyantais
35627                      Les-esclaves-antiques,-ancêtres-de-nos-robots-?
Name: title, Length: 47175, dtype: object

In [94]:
# cleaning the values
corpus_complet['title_clean'] = corpus_complet['title'].str.replace("-", ' ')
corpus_complet['title_clean'] = corpus_complet['title_clean'].str.replace("[", '')
corpus_complet['title_clean'] = corpus_complet['title_clean'].str.replace("]", '')

<ipython-input-94-24a0dc1c5f83>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-94-24a0dc1c5f83>:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.

<ipython-input-94-24a0dc1c5f83>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-94-24a0dc1c5f83>:3: FutureWarning:

The default value of regex will change from True to False in 

### Unique ID column

In [95]:
# creating unique id values
n = []
for i in corpus_complet.index:
    n.append(random.randint(10000,100000))
corpus_complet['unique_id'] = n

<ipython-input-95-3c478cb79724>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Final dataframe

In [96]:
# creating final dataframe
corpus_final = corpus_complet[['unique_id', 'author','journal_clean', 'content', 'date_time', 'title_clean', 'key_words']]

In [98]:
print('Total number of records in the final dataset:', corpus_final.shape)

Total number of records in the final dataset: (47175, 7)


## Text cleaning

In [99]:
# transfroming articles to string
corpus_final['text_as_string'] = corpus_final['content'].astype(str) 

<ipython-input-99-159b64a7d4e9>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [100]:
# creating stop word list
stop_words = open('stopwords-fr.txt','r', encoding="utf-8").read().split('\n')

In [101]:
# creating the function for text cleaning
def clean_text(text):
    text=unidecode(text) # remove accents of text
    text=text.lower() # lowercase
    text = re.sub(r'\d+', '', text) # remove numbers
    text=re.sub(r'[^\sa-zA-Z0-9@\[\]]',' ', text) # removing characters: punctuation and other special characters
    text=re.sub(r"https*\S+", '', text) # removing links
    text = re.sub(r"\sd\s", " ", text) # removing single letters
    text = re.sub(r"\sf\s", " ", text)
    text = re.sub(r"\si\s", " ", text)
    text = re.sub(r"\sm\s", " ", text)
    text = re.sub(r"\ss\s", " ", text)
    text = re.sub(r"\st\s", " ", text)
    text = re.sub(r"\su\s", " ", text)
    text = re.sub(r"\sy\s", " ", text)
    text = re.sub(r"\sl\s", " ", text)
    text = re.sub(r"\sc\s", " ", text)
    text = re.sub(r"\sa\s", " ", text)
    text = re.sub(r"\sn\s", " ", text)
    text = re.sub(r"\sr\s", " ", text)
    text = re.sub(r"\se\s", " ", text)
    text = ' '.join([word for word in text.split(' ') if word not in stop_words]) # removing stopwords
    text = " ".join([word for word in text.split()]) # removing additional space
    return text

In [102]:
# applying function 
corpus_final['cleaned_text'] = corpus_final.text_as_string.apply(lambda x: clean_text(x))
corpus_final['cleaned_text'].head(20)

<ipython-input-102-072cbf889ac6>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



198    supplement digital nouvelle star xxie siecle elon musk croit capable declencher troisieme guerre mondiale environnement medecine armement partout intelligence artificielle affole partout imperturbable microcosme creation publicitaire putsch demain president betc stephane xiberras exemple question effets annonce joue amalgame veritable intelligence artificielle capable penser automatisation taches analyse donnees instant jamais permis creer constat approuve francois brogi vp creation agence data artefact crois pouvoir emotion surprise audace traits ia suffisamment dotee puisqu base apprentissage existant jean philippe desbiolles vice president ibm watson tranche question considerant saut creatif bel humain actuellement maturite agir systemes supervises crois complementarite createur ia devenant formidable stimulant maniere repondre besoins hyperpersonnalisation consommateur crise confiance outils soutien ia generer acte creatif soutenir aide effective ensemble chaine commencer in

## Visualization of the word usage

In [111]:
# creating text string
mylist = []
for string in corpus_final['cleaned_text']:
    mylist.append(string)
mylist

text = str(mylist)

In [112]:
# tokenization
txt_tokens = word_tokenize(text)
txt_tokens = [word.lower() for word in txt_tokens if word.isalpha()]
txt_tokens = [word for word in txt_tokens if not word in stop_words]

In [123]:
# creation of the bigrams dataframe
bigrams_series = (pd.Series(nltk.ngrams(txt_tokens, 2)).value_counts())
bigrams = pd.DataFrame(bigrams_series.sort_values(ascending=False))
bigrams = bigrams.reset_index().rename(columns={'index': 'bigram', 0:'count'})
bigrams['bigram'] = bigrams['bigram'].astype(str)
bigrams['bigram'] = bigrams['bigram'].str.replace("(", '')
bigrams['bigram'] = bigrams['bigram'].str.replace(")", '')
bigrams['bigram'] = bigrams['bigram'].str.replace(",", '_')
bigrams['bigram'] = bigrams['bigram'].str.replace(" ", '')
bigrams['bigram'] = bigrams['bigram'].str.replace("'", '')

<ipython-input-123-320d46036b7d>:6: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.

<ipython-input-123-320d46036b7d>:7: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



In [125]:
# deleting non meaningful bigrams
bigrams = bigrams.drop([bigrams.index[0], bigrams.index[2], bigrams.index[3], bigrams.index[5], bigrams.index[7], bigrams.index[17]])

In [127]:
# bigrams visualisation
fig = px.bar(bigrams[:20], x='count', y='bigram', title='Counts of top bigrams', template='plotly_white')
fig.update_xaxes(title_text='Words count')
fig.update_yaxes(title_text='Bigram')
fig.show()

In [128]:
# saving data for dashboard
bigrams.to_csv('dist_bigram.csv', index=False)